In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import tensorflow_datasets as tfds
import re
import random

In [2]:
data = tfds.load('tiny_shakespeare', split='train')
text = ""
for d in data.take(1):
    text = d['text'].numpy().decode('utf-8')

In [3]:
print("Sample text preview:\n", text[:500])

Sample text preview:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
text = text.lower()
text = re.sub(r'[^a-zA-Z\s]', '', text)

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print("Total unique words:", total_words)

Total unique words: 12072


In [6]:
input_sequences = []
token_list = tokenizer.texts_to_sequences([text])[0]
seq_length = 20

In [7]:
for i in range(seq_length, len(token_list)):
    seq = token_list[i-seq_length:i+1]
    input_sequences.append(seq)

In [8]:
print("Total sequences:", len(input_sequences))

Total sequences: 182447


In [9]:
input_sequences = np.array(input_sequences)
X = input_sequences[:,:-1]
y = input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [10]:
print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (182447, 20)
Output shape: (182447, 12072)


In [11]:
model = Sequential([
    Input(shape=(seq_length,)),
    Embedding(total_words, 100),
    LSTM(150, return_sequences=True),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

In [12]:
# from tensorflow.keras.models import Sequential
# model.build(input_shape=(None, seq_length))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 100)        │     1,207,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12072)          │     1,219,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,677,472 (10.21 MB)

 Trainable params: 2,677,472 (10.21 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# model.build(input_shape=(None, seq_length))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='loss', patience=3)
checkpoint = ModelCheckpoint('best_lstm_model.keras', save_best_only=True, monitor='loss')



In [ ]:

print("Starting training...")
try:
    history = model.fit(X, y, epochs=30, batch_size=256, callbacks=[early_stop, checkpoint])
    print("✓ Training complete!")
except Exception as e:
    print(f"✗ Error during training: {e}")
    raise

Starting training...
